In [1]:
import numpy
import optuna
from pycbc.filter import matched_filter
from pycbc.waveform import get_td_waveform
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation

/home/susanna.green/.conda/envs/PyCBCandOptuna/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sorting_the_mass(m1, m2):
    if m1>m2:
        return m1, m2
    else:
        return m2, m1

In [3]:
merger = Merger("GW150914") #GW150914

# Get the data from the Hanford detector
strain = merger.strain('H1')

# Remove the low frequency content and downsample the data to 2048Hz
strain = resample_to_delta_t(highpass(strain, 15.0), 1.0/2048)

# Remove 2 seconds of data from both the beginning and end
conditioned = strain.crop(2, 2)

# Estimate the power spectral density
# We use 4 second samles of our time series in Welch method.
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner

psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to informat the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),low_frequency_cutoff=15)

In [6]:
#Optuna gnerating the template. 
def objective(trial):
    m1 = trial.suggest_float('m1', 10, 100)
    m2 = trial.suggest_float('m2', 10, 100)
    m1, m2 = sorting_the_mass(m1, m2)
    s1 = trial.suggest_float('s1', -0.99, 0.99)
    s2 = trial.suggest_float('s2', -0.99, 0.99)
    hp, hc = get_td_waveform(approximant="IMRPhenomXAS", mass1=m1, mass2=m2, spin1z=s1, spin2z=s2, delta_t=conditioned.delta_t,f_lower=20)
    hp.resize(len(conditioned))
    template = hp.cyclic_time_shift(hp.start_time)
    snr = matched_filter(template, conditioned, psd=psd, low_frequency_cutoff=20)
    snr = snr.crop(4 + 4, 4)
    peak = abs(snr).numpy().argmax()
    snrp = abs(snr[peak])
    return float(snrp)

In [7]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="maximize")
study.optimize(objective, n_trials=1000)
print(study.best_params)

[I 2023-03-01 20:18:43,671] A new study created in memory with name: no-name-2d916273-e201-46ad-b13c-d0b5f3613f6a
[I 2023-03-01 20:18:43,766] Trial 0 finished with value: 13.582000645658223 and parameters: {'m1': 86.76046488114049, 'm2': 14.105836315450855, 's1': 0.5238855502621189, 's2': 0.15809372970370061}. Best is trial 0 with value: 13.582000645658223.
[I 2023-03-01 20:18:43,821] Trial 1 finished with value: 15.687821191571857 and parameters: {'m1': 56.421915702968626, 'm2': 50.39176064103719, 's1': 0.6254059004102146, 's2': -0.3404331666102599}. Best is trial 1 with value: 15.687821191571857.
[I 2023-03-01 20:18:43,876] Trial 2 finished with value: 11.444770988497805 and parameters: {'m1': 26.759102199160044, 'm2': 89.5188324932577, 's1': -0.6434148738690906, 's2': 0.14690013082239273}. Best is trial 1 with value: 15.687821191571857.
[I 2023-03-01 20:18:43,934] Trial 3 finished with value: 17.610072116611274 and parameters: {'m1': 54.08948545139607, 'm2': 26.628803857423915, 's1'

{'m1': 39.89603541063972, 'm2': 29.84915757134386, 's1': -0.18016076848481433, 's2': -0.024179244222583565}
